In [1]:
import numpy as np
import pandas as pd
from faker import Faker
import random
from datetime import datetime, timedelta

# Configuración inicial
fake = Faker('es_ES')

# Esta función establece una "semilla" para la generación de números aleatorios.
def set_seed(seed):
    np.random.seed(seed)
    random.seed(seed)

# Generación de sueldo basado en el grupo del usuario
def generar_sueldo(grupo, grupos_caracteristicas):
    return round(np.random.uniform(*grupos_caracteristicas[grupo]['sueldo']), 2)

# Cálculo del dinero en banco inicial (en miles ahora)
def calcular_dinero_banco(grupo, grupos_caracteristicas):
    return round(np.random.uniform(*grupos_caracteristicas[grupo]['dinero_banco']), 2)

# Asigna un grupo al usuario basado en las probabilidades definidas
def asignar_grupo(probabilidades_grupos):
    return np.random.choice(list(probabilidades_grupos.keys()), p=list(probabilidades_grupos.values()))

# Genera un IBAN (número de cuenta bancaria) para Kutxabank
def generar_iban_kutxabank():
    entidad = "2095"
    oficina = f"{random.randint(0, 9999):04d}"
    dc = f"{random.randint(0, 99):02d}"
    cuenta = f"{random.randint(0, 9999999999):010d}"
    return f"ES95{entidad}{oficina}{dc}{cuenta}"

# Genera una edad aleatoria dentro del rango especificado
def generar_edad(rango_edad):
    return random.randint(rango_edad[0], rango_edad[1])

# Genera un número aleatorio de hijos basado en el grupo del usuario
def generar_hijos(grupo, grupos_caracteristicas):
    min_hijos, max_hijos = grupos_caracteristicas[grupo]['hijos']
    return random.randint(min_hijos, max_hijos)

# Genera el sexo del usuario basado en la distribución real por edad
def generar_sexo():
    return np.random.choice(['H', 'M'])

# Genera un nombre para el usuario basado en su sexo
def generar_nombre(sexo):
    return fake.first_name_male() if sexo == 'H' else fake.first_name_female()

# Genera unos apellidos para el usuario
def generar_apellidos():
    return f"{fake.last_name()} {fake.last_name()}"

# Definir objetivos de ahorro según el grupo
def definir_objetivos_ahorro(grupo):
    if grupo == 'Grupo 1':
        return [
            {'objetivo': 'Casa Grande', 'monto_objetivo': 50000, 'fecha_limite': '2026-12-01'},
            {'objetivo': 'Universidad Hijos', 'monto_objetivo': 30000, 'fecha_limite': '2035-09-01'},
            {'objetivo': 'Fondo de Emergencia', 'monto_objetivo': 10000, 'fecha_limite': '2024-12-31'}
        ]
    elif grupo == 'Grupo 2':
        return [
            {'objetivo': 'Vacaciones', 'monto_objetivo': 5000, 'fecha_limite': '2025-06-01'},
            {'objetivo': 'Coche', 'monto_objetivo': 15000, 'fecha_limite': '2026-06-01'}
        ]
    elif grupo == 'Grupo 3':
        return [
            {'objetivo': 'Reformas del Piso', 'monto_objetivo': 20000, 'fecha_limite': '2026-01-01'},
            {'objetivo': 'Fondo de Emergencia', 'monto_objetivo': 10000, 'fecha_limite': '2024-12-31'}
        ]
    elif grupo == 'Grupo 4':
        return [
            {'objetivo': 'Casa Propia', 'monto_objetivo': 60000, 'fecha_limite': '2027-01-01'},
            {'objetivo': 'Coche Familiar', 'monto_objetivo': 20000, 'fecha_limite': '2025-01-01'}
        ]

# Genera transacciones de cuenta corriente (ingresos y gastos)
def generar_transacciones_corriente(usuario_id, sueldo, gastos_fijos, meses=12):
    transacciones = []
    for mes in range(1, meses + 1):
        # Ingreso mensual
        transacciones.append({
            'usuario_id': usuario_id,
            'fecha': f'2024-{mes:02d}-01',
            'tipo': 'Ingreso',
            'monto': sueldo
        })
        
        # Gastos mensuales
        for gasto in gastos_fijos:
            transacciones.append({
                'usuario_id': usuario_id,
                'fecha': f'2024-{mes:02d}-10',
                'tipo': gasto['tipo'],
                'monto': -gasto['monto']
            })
    
    return transacciones

# Genera transacciones de ahorro después de gastos mensuales si hay saldo disponible
def generar_transacciones_ahorro(usuario_id, objetivos_ahorro, saldo_disponible):
    transacciones_ahorro = []
    if saldo_disponible > 0:  # Ahorra solo si hay saldo disponible
        for objetivo in objetivos_ahorro:
            ahorro_mensual = round(saldo_disponible * 0.2, 2)  # Transferir el 20% del saldo disponible
            if ahorro_mensual > 0:
                transacciones_ahorro.append({
                    'usuario_id': usuario_id,
                    'fecha': datetime.now().strftime('%Y-%m-%d'),
                    'objetivo': objetivo['objetivo'],
                    'monto': min(ahorro_mensual, objetivo['monto_objetivo'])  # Asegurarse de no transferir más de lo necesario
                })
                saldo_disponible -= ahorro_mensual
    return transacciones_ahorro

# Función principal para generar usuarios con características avanzadas y objetivos de ahorro
def generar_usuarios_avanzados(n, grupos_caracteristicas, probabilidades_grupos, rango_edad, seed=None):
    if seed is not None:
        set_seed(seed)
    
    usuarios = []
    objetivos_ahorro = []
    transacciones_corriente = []
    transacciones_ahorro = []
    ibans_generados = set()
    
    for user_id in range(1, n+1):
        grupo = asignar_grupo(probabilidades_grupos)
        edad = generar_edad(rango_edad)
        sexo = generar_sexo()
        
        usuario = {
            'id': user_id,
            'nombre': generar_nombre(sexo),
            'apellidos': generar_apellidos(),
            'edad': edad,
            'sexo': sexo,
            'grupo': grupo,
            'sueldo': generar_sueldo(grupo, grupos_caracteristicas),
            'dinero_banco': calcular_dinero_banco(grupo, grupos_caracteristicas),
            'vivienda': 'Propiedad' if random.random() < grupos_caracteristicas[grupo]['porcentaje_vivienda'] else 'Alquiler',
            'coche': random.random() < grupos_caracteristicas[grupo]['porcentaje_coche'],
            'hijos': generar_hijos(grupo, grupos_caracteristicas),
            'iban': generar_iban_kutxabank()
        }
        
        while usuario['iban'] in ibans_generados:
            usuario['iban'] = generar_iban_kutxabank()
        ibans_generados.add(usuario['iban'])
        
        # Añadir objetivos de ahorro
        objetivos = definir_objetivos_ahorro(grupo)
        for objetivo in objetivos:
            objetivos_ahorro.append({
                'usuario_id': user_id,
                'objetivo': objetivo['objetivo'],
                'monto_objetivo': objetivo['monto_objetivo'],
                'fecha_limite': objetivo['fecha_limite'],
                'monto_ahorrado': 0  # Inicialmente 0
            })
        
        # Generar gastos fijos para el usuario (Ejemplo: alquiler, coche, etc.)
        gastos_fijos = [
            {'tipo': 'Alquiler' if usuario['vivienda'] == 'Alquiler' else 'Hipoteca', 'monto': random.uniform(400, 800)},
            {'tipo': 'Comida', 'monto': random.uniform(300, 500)},
            {'tipo': 'Transporte', 'monto': random.uniform(100, 300)}
        ]
        
        # Generar transacciones de cuenta corriente (gastos e ingresos)
        transacciones = generar_transacciones_corriente(user_id, usuario['sueldo'], gastos_fijos)
        transacciones_corriente.extend(transacciones)
        
        # Calcular el saldo disponible después de los gastos y generar transacciones de ahorro
        ingresos_totales = usuario['sueldo'] * 12  # Supongamos ingresos mensuales regulares
        gastos_totales = sum([gasto['monto'] * 12 for gasto in gastos_fijos])  # Gastos anuales
        saldo_disponible = ingresos_totales - gastos_totales
        
        # Solo generar transacciones de ahorro si hay saldo disponible
        if saldo_disponible > 0:
            transacciones_ahorro_mes = generar_transacciones_ahorro(user_id, objetivos, saldo_disponible)
            transacciones_ahorro.extend(transacciones_ahorro_mes)
        
        usuarios.append(usuario)
    
    return pd.DataFrame(usuarios), pd.DataFrame(objetivos_ahorro), pd.DataFrame(transacciones_corriente), pd.DataFrame(transacciones_ahorro)

# Características de cada grupo basadas en la tabla proporcionada
grupos_caracteristicas = {
    'Grupo 1': {
        'sueldo': (2800, 4600),
        'dinero_banco': (10000, 14000),  # Ajustado para estar en miles
        'porcentaje_vivienda': 0.9,
        'porcentaje_coche': 1,
        'hijos': (1, 3),
    },
    'Grupo 2': {
        'sueldo': (1300, 2100),
        'dinero_banco': (100, 4000),  # Ajustado para estar en miles
        'porcentaje_vivienda': 0.1,
        'porcentaje_coche': 0.5,
        'hijos': (0, 1),
    },
    'Grupo 3': {
        'sueldo': (1300, 2100),
        'dinero_banco': (6000, 14000),  # Ajustado para estar en miles
        'porcentaje_vivienda': 0.7,
        'porcentaje_coche': 0.3,
        'hijos': (0, 1),
    },
    'Grupo 4': {
        'sueldo': (2800, 4600),
        'dinero_banco': (2000, 10000),  # Ajustado para estar en miles
        'porcentaje_vivienda': 0.2,
        'porcentaje_coche': 0.5,
        'hijos': (1, 2),
    }
}

# Probabilidades para asignar grupos (equiprobable en este caso)
probabilidades_grupos = {
    'Grupo 1': 0.25,
    'Grupo 2': 0.25,
    'Grupo 3': 0.25,
    'Grupo 4': 0.25
}

# Rango de edad
rango_edad = (25, 40)

# Generar usuarios, objetivos de ahorro, transacciones de cuenta corriente y transacciones de ahorro
usuarios_df, objetivos_ahorro_df, transacciones_corriente_df, transacciones_ahorro_df = generar_usuarios_avanzados(10, grupos_caracteristicas, probabilidades_grupos, rango_edad, seed=42)

# Guardar los DataFrames en archivos CSV
usuarios_df.to_csv('usuarios.csv', index=False)
objetivos_ahorro_df.to_csv('objetivos_ahorro.csv', index=False)
transacciones_corriente_df.to_csv('transacciones_corriente.csv', index=False)
transacciones_ahorro_df.to_csv('transacciones_ahorro.csv', index=False)

# Mostrar las primeras filas de cada tabla
print("Usuarios:")
print(usuarios_df.head())
print("\nObjetivos de Ahorro:")
print(objetivos_ahorro_df.head())
print("\nTransacciones de Cuenta Corriente:")
print(transacciones_corriente_df.head())
print("\nTransacciones de Ahorro:")
print(transacciones_ahorro_df.head())


Usuarios:
   id    nombre       apellidos  edad sexo    grupo   sueldo  dinero_banco  \
0   1   Marcelo  Castro Jiménez    28    H  Grupo 2  1446.75       3140.79   
1   2   Marisol   Herrero Múñiz    26    M  Grupo 3  1424.80       6464.67   
2   3  Elisabet   Caparrós Arco    30    M  Grupo 4  3057.16       7207.11   
3   4     Adora     Rius Robles    33    M  Grupo 1  4298.40      10849.36   
4   5      Amor    Puga Peralta    36    H  Grupo 1  3911.47      12446.61   

    vivienda  coche  hijos                      iban  
0  Propiedad   True      0  ES9520952286949030148007  
1  Propiedad   True      0  ES9520959195251801823908  
2   Alquiler   True      1  ES9520953527971445662585  
3  Propiedad   True      3  ES9520952045488928378856  
4  Propiedad   True      1  ES9520953733984728765136  

Objetivos de Ahorro:
   usuario_id             objetivo  monto_objetivo fecha_limite  \
0           1           Vacaciones            5000   2025-06-01   
1           1                Coche 

: 